In [58]:
from io import TextIOWrapper
import plotly.express as px
import plotly.graph_objects as go
import plotly as plt
import pandas as pd
import os

In [59]:
def parse(file: TextIOWrapper):
    n = file.readline()[:-2]
    values = [float(file.readline()) for _ in range(2)]
    return n, values

In [60]:
def create_scatter(files, N = 20):
    x = []
    y = []
    params = []
    for filename, tag in files:
        with open(f"{os.getcwd()}/samples/{filename}") as file:
            for _ in range(N):
                n, values = parse(file)

                x.append(n)
                y.append(sum(values) / len(values))

                params.append(tag)


    df = pd.DataFrame(data={"N": x, "Time": y, "Params": params})
    return px.scatter(df, x="N", y="Time", color="Params").update_coloraxes(
        showscale=False
    )
    
# .update_layout(showlegend=False).show()

# Эксперимент 1

## Алгоритм 1
```c
#include <stdlib.h>
#include <stdio.h>

int main(int argc, char const *argv[])
{
    /* ... */
    long long N = atoll(argv[1]);
    long long R = 0;
    for (long long i = 0; i < N; ++i) {
        R += i;
    }

    printf("%Ld", R);
}

```

Программа считает сумму чисел до `N`.


## 1 Процессор

In [61]:
create_scatter(
    [
        ("samples-1-seq.txt", "1 Processor sequential"),
        ("samples-1-par.txt", "1 Processor parallel"),
    ]
).show()

### Вывод
При одном процессоре в системе параллельное и последовательное исполнения, не отличаются, что не странно т.к оба процесса CPU-bound и мы в обоих случаях используем CPU на максимум.

## 2 Процессора

In [62]:
create_scatter(
    [
        ("samples-2-seq.txt", "2 Processors sequential"),
        ("samples-2-par.txt", "2 Processors parallel"),
    ]
).show()

### Вывод
Когда процессоров становится два, то разница между параллельным и последовательным исполенением становится примерно в 2 раза. Теперь когда в системе есть два CPU-bound процесса, которые никак не связаны, то они могут исполнятся одновременно без каких-либо проблем. Отсюда и уменьшение времени в 2 раза.  

# Эксперимент 2

## Алгоритм 2

```c
#include <stdlib.h>
#include <stdio.h>

int main(int argc, char const *argv[])
{
    /* ... */
    FILE* f_read = fopen(argv[1], "r");
    FILE* f_append = fopen(argv[1], "a");
    /* ... */
    for (size_t i = 1; i <= n; i++)
    {
        int v=55;
        fscanf(f_read, "%d", &v);
        v *= 2;
        fprintf(f_append, "%d\n", v);
    }
    
    fclose(f_append);
    fclose(f_read);
}
```

Программа читает числа из файла, умножает их на 2 и записывает в конец того же файла.


## 1 Процессор

In [70]:
create_scatter(
    [
        ("fsamples-1-seq.txt", "1 Processors sequential"),
        ("fsamples-1-par.txt", "1 Processors parallel"),
    ],
17).show()

### Вывод
Как и в первом эксперименте, запуск параллельно не уменьшая общее время исполнения. Каждый скрипт работает со своим файлом и т.к `IO` буфферизировано на уровне ОС, то упор оказывается снова в CPU, т.к процессы не конкурируют ни за какой другой ресурс.

## 2 Процессора

In [69]:
create_scatter(
    [
        ("fsamples-2-seq.txt", "2 Processors sequential"),
        ("fsamples-2-par.txt", "2 Processors parallel"),
    ],
17).show()

### Вывод
В случае с двумя процессорами мы опять видим ускорение примерно в две раза. Как упоминалось в прошлом пункте, что единственный разделяемый ресурс оказывается CPU. Так же я уже упоминал, что ввод вывод буфферизован на уровне ОС, так что взаимодействия с диском минимизированы.